In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn import compose
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [2]:
df = pd.read_csv('instacart.csv')
df.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,Organic Egg Whites,86,16,eggs,dairy eggs,202279,prior,3,5,9,8.0
1,2,28985,2,1,Michigan Organic Kale,83,4,fresh vegetables,produce,202279,prior,3,5,9,8.0
2,2,9327,3,0,Garlic Powder,104,13,spices seasonings,pantry,202279,prior,3,5,9,8.0
3,2,45918,4,1,Coconut Butter,19,13,oils vinegars,pantry,202279,prior,3,5,9,8.0
4,2,30035,5,0,Natural Sweetener,17,13,baking ingredients,pantry,202279,prior,3,5,9,8.0


In [24]:
#df[:100].groupby('user_id').agg({''})

In [19]:
df_c=df[:20000]

In [27]:
df_c[df_c.eval_set=='train']

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
32892291,1121100,28373,2,0,Original Rotisserie Chicken,13,20,prepared meals,deli,170368,train,18,3,14,6.0
33152090,1763602,16797,12,0,Strawberries,24,4,fresh fruits,produce,169439,train,23,0,7,29.0
32666927,568601,4962,4,1,Yotoddler Organic Pear Spinach Mango Yogurt,120,16,yogurt,dairy eggs,4945,train,5,1,9,30.0
33656852,3020666,27104,6,1,Fresh Cauliflower,83,4,fresh vegetables,produce,76919,train,12,5,11,10.0
33325220,2197524,7500,4,1,Organic Vanilla Extract,17,13,baking ingredients,pantry,46149,train,4,5,11,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32671029,577420,31196,16,1,Half Dozen Grade A White Eggs Large,86,16,eggs,dairy eggs,14392,train,29,2,10,21.0
32966475,1306413,42701,10,0,Organic Sour Cream,108,16,other creams cheeses,dairy eggs,149825,train,10,3,9,7.0
32921418,1195644,43772,14,0,Cherubs Heavenly Salad Tomatoes,83,4,fresh vegetables,produce,167515,train,17,1,13,30.0
32783223,850181,49131,2,1,Smoked Turkey Breast Slices,96,20,lunch meat,deli,199305,train,66,1,13,4.0


In [26]:
df_c.eval_set.unique()

array(['prior', 'train'], dtype=object)

In [36]:
X_train=df_c.loc[(df.eval_set == 'prior')][['add_to_cart_order','reordered','order_number','user_id',
                                          'order_dow','order_hour_of_day','days_since_prior_order']]
y_train=df_c.loc[(df.eval_set == 'prior')]['department_id']

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn import impute
from sklearn import preprocessing

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules 

In [37]:
def make_pipelines():
    """Create a pipeline for each of the following algorithms:
    1. Logistic Regression
    2. k-nearest neighbors (KNN) 
    3. Naive Bayes (Guassian)
    4. Support Vector Machines (SVM)
    5. Random Forest™ 
    
    If appropriate, apply StandardScaler before the algorithm.   
    Use default hyperparameters.
    If an algorithm takes random_state then random_state=42 
    
    Return a list of all the pipelines.
    """ 
    numeric_features=['reordered','order_number','user_id',
                                          'order_dow','order_hour_of_day','days_since_prior_order','add_to_cart_order']
    numeric_transformer = Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='median'))])
    
    preprocessor = compose.ColumnTransformer(transformers=[
        ('num', numeric_transformer, numeric_features)])
    
    LR=Pipeline([('preprocessor', preprocessor),('LogisticRegression', LogisticRegression(random_state=42))])
    KNN=Pipeline([('preprocessor', preprocessor),('scaler', StandardScaler()),('KNeighborsClassifier', KNeighborsClassifier())])
    NB=Pipeline([('preprocessor', preprocessor),('GaussianNB', GaussianNB())])
    SVM=Pipeline([('preprocessor', preprocessor),('scaler', StandardScaler()), ('SVC', SVC(random_state=42))])
    RF=Pipeline([ ('preprocessor', preprocessor),('RandomForestClassifier', RandomForestClassifier(random_state=42))])
    
    

    pipelines=[LR,KNN,NB,SVM,RF]
    return pipelines

In [38]:
pipelines = make_pipelines()

In [39]:
for pipe in pipelines:
    pipe.fit(X_train, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [40]:
from sklearn.metrics import classification_report

for pipe in pipelines:
    print(pipe.steps[-1][0],'\n',classification_report(y_train, pipe.predict(X_train)))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


LogisticRegression 
               precision    recall  f1-score   support

           1       0.00      0.00      0.00      1374
           2       0.00      0.00      0.00        11
           3       0.00      0.00      0.00       672
           4       0.29      1.00      0.45      5646
           5       0.00      0.00      0.00       100
           6       0.00      0.00      0.00       145
           7       0.00      0.00      0.00      1611
           8       0.00      0.00      0.00        58
           9       0.00      0.00      0.00       491
          10       0.00      0.00      0.00        17
          11       0.00      0.00      0.00       246
          12       0.00      0.00      0.00       399
          13       0.00      0.00      0.00      1178
          14       0.00      0.00      0.00       394
          15       0.00      0.00      0.00       690
          16       0.00      0.00      0.00      3080
          17       0.00      0.00      0.00       440
      

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


KNeighborsClassifier 
               precision    recall  f1-score   support

           1       0.28      0.50      0.36      1374
           2       0.00      0.00      0.00        11
           3       0.25      0.33      0.28       672
           4       0.44      0.78      0.56      5646
           5       0.17      0.05      0.08       100
           6       0.18      0.06      0.09       145
           7       0.40      0.24      0.30      1611
           8       0.50      0.07      0.12        58
           9       0.39      0.12      0.18       491
          10       0.00      0.00      0.00        17
          11       0.34      0.05      0.09       246
          12       0.28      0.06      0.10       399
          13       0.42      0.25      0.32      1178
          14       0.41      0.08      0.13       394
          15       0.43      0.14      0.21       690
          16       0.46      0.37      0.41      3080
          17       0.45      0.08      0.14       440
    

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
